In [1]:
pip install pandas scikit_learn gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
#1_Data_Loading_n_Preprocessing:

import pandas as pd

# Load the datasets
train_features = pd.read_csv('train.csv')
train_labels = pd.read_csv('trainLabels.csv')

# Inspect the data
print("Train Features Shape:", train_features.shape)
print("Train Labels Shape:", train_labels.shape)
print(train_features.head())
print(train_labels.head())

Train Features Shape: (9999, 146)
Train Labels Shape: (49999, 34)
   id   x1   x2                                            x3  \
0   1   NO   NO  dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=   
1   2  NaN  NaN                                           NaN   
2   3   NO   NO  ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=   
3   4  YES   NO  BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=   
4   5   NO   NO  RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=   

                                             x4        x5        x6        x7  \
0  GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=  0.576561  0.073139  0.481394   
1                                           NaN  0.000000  0.000000  0.000000   
2  X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=  1.341803  0.051422  0.935572   
3  YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=  0.653912  0.041471  0.940787   
4  3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=  1.415919  0.000000  1.000000   

         x8        x9  ... x136   x137  x138  x139 x140 

In [4]:
# Check for missing values
print(train_features.isnull().sum())
print(train_labels.isnull().sum())

# Basic data statistics
print(train_features.describe())
print(train_labels.describe())


id         0
x1      1426
x2      1426
x3      1426
x4      1426
        ... 
x141       0
x142       0
x143       0
x144       0
x145       0
Length: 146, dtype: int64
id     0
y1     0
y2     0
y3     0
y4     0
y5     0
y6     0
y7     0
y8     0
y9     0
y10    0
y11    0
y12    0
y13    0
y14    0
y15    0
y16    0
y17    0
y18    0
y19    0
y20    0
y21    0
y22    0
y23    0
y24    0
y25    0
y26    0
y27    0
y28    0
y29    0
y30    0
y31    0
y32    0
y33    0
dtype: int64
                id           x5           x6           x7           x8  \
count  9999.000000  9999.000000  9999.000000  9999.000000  9999.000000   
mean   5000.000000     0.964933     0.053623     0.791668     0.172705   
std    2886.607005     0.524688     0.128503     0.353194     0.329020   
min       1.000000     0.000000     0.000000     0.000000     0.000000   
25%    2500.500000     0.642796     0.000000     0.831299     0.000000   
50%    5000.000000     1.285541     0.000000     0.960637     0.0000

In [5]:
# Handle missing values
# For simplicity, let's fill missing numerical values with the mean and categorical with the mode

for column in train_features.columns:
    if train_features[column].dtype == 'object':
        train_features[column].fillna(train_features[column].mode()[0], inplace=True)
    else:
        train_features[column].fillna(train_features[column].mean(), inplace=True)

# Verify that there are no more missing values
print(train_features.isnull().sum())


id      0
x1      0
x2      0
x3      0
x4      0
       ..
x141    0
x142    0
x143    0
x144    0
x145    0
Length: 146, dtype: int64


/var/folders/g0/5hmfbqr17y12_gxw5ps7slkw0000gp/T/ipykernel_90261/2011777375.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_features[column].fillna(train_features[column].mean(), inplace=True)
/var/folders/g0/5hmfbqr17y12_gxw5ps7slkw0000gp/T/ipykernel_90261/2011777375.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

In [6]:
from sklearn.preprocessing import OneHotEncoder

# Identify categorical and numerical features
categorical_features = train_features.select_dtypes(include=['object']).columns
numerical_features = train_features.select_dtypes(include=['number']).columns

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_features = pd.DataFrame(encoder.fit_transform(train_features[categorical_features]))

# Fix the column names for the encoded features
encoded_categorical_features.columns = encoder.get_feature_names_out(categorical_features)

# Combine numerical and encoded categorical features
processed_features = pd.concat([train_features[numerical_features].reset_index(drop=True), 
                                encoded_categorical_features.reset_index(drop=True)], axis=1)

print("Processed Features Shape:", processed_features.shape)
print(processed_features.head())


Processed Features Shape: (9999, 34855)
   id        x5        x6        x7        x8        x9  x15       x16  x17  \
0   1  0.576561  0.073139  0.481394  0.115697  0.472474   42  0.396065    3   
1   2  0.000000  0.000000  0.000000  0.000000  0.000000    0  0.000000    0   
2   3  1.341803  0.051422  0.935572  0.041440  0.501710    2  0.838475    3   
3   4  0.653912  0.041471  0.940787  0.090851  0.556564   37  0.127405    8   
4   5  1.415919  0.000000  1.000000  0.000000  0.375297    1  0.523543    4   

   x18  ...  x129_NO  x129_YES  x130_NO  x130_YES  x140_NO  x140_YES  x141_NO  \
0    6  ...      1.0       0.0      1.0       0.0      0.0       1.0      1.0   
1    0  ...      1.0       0.0      1.0       0.0      0.0       1.0      1.0   
2    5  ...      1.0       0.0      1.0       0.0      1.0       0.0      1.0   
3   15  ...      1.0       0.0      1.0       0.0      1.0       0.0      1.0   
4   11  ...      1.0       0.0      1.0       0.0      1.0       0.0      1.0   

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load the datasets
train_features = pd.read_csv('train.csv')
train_labels = pd.read_csv('trainLabels.csv')

# Inspect the data
print("Train Features Shape:", train_features.shape)
print("Train Labels Shape:", train_labels.shape)
print(train_features.head())
print(train_labels.head())

# Check for missing values
print(train_features.isnull().sum())
print(train_labels.isnull().sum())

# Handle missing values
for column in train_features.columns:
    if train_features[column].dtype == 'object':
        train_features[column].fillna(train_features[column].mode()[0], inplace=True)
    else:
        train_features[column].fillna(train_features[column].mean(), inplace=True)

# Verify that there are no more missing values
print(train_features.fillna(0).sum())

# Identify categorical and numerical features
categorical_features = train_features.select_dtypes(include=['object']).columns
numerical_features = train_features.select_dtypes(include=['number']).columns

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_categorical_features = pd.DataFrame(encoder.fit_transform(train_features[categorical_features]))

# Fix the column names for the encoded features
encoded_categorical_features.columns = encoder.get_feature_names_out(categorical_features)

# Combine numerical and encoded categorical features
processed_features = pd.concat([train_features[numerical_features].reset_index(drop=True), 
                                encoded_categorical_features.reset_index(drop=True)], axis=1)

print("Processed Features Shape:", processed_features.shape)
print(processed_features.head())


Train Features Shape: (9999, 146)
Train Labels Shape: (49999, 34)
   id   x1   x2                                            x3  \
0   1   NO   NO  dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=   
1   2  NaN  NaN                                           NaN   
2   3   NO   NO  ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=   
3   4  YES   NO  BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=   
4   5   NO   NO  RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=   

                                             x4        x5        x6        x7  \
0  GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=  0.576561  0.073139  0.481394   
1                                           NaN  0.000000  0.000000  0.000000   
2  X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=  1.341803  0.051422  0.935572   
3  YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=  0.653912  0.041471  0.940787   
4  3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=  1.415919  0.000000  1.000000   

         x8        x9  ... x136   x137  x138  x139 x140 

/var/folders/g0/5hmfbqr17y12_gxw5ps7slkw0000gp/T/ipykernel_90261/1288792104.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_features[column].fillna(train_features[column].mean(), inplace=True)
/var/folders/g0/5hmfbqr17y12_gxw5ps7slkw0000gp/T/ipykernel_90261/1288792104.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whic

id                                               49995000
x1      NONONOYESNONONONONONONOYESNOYESNONONONONONONON...
x2      NONONONONONONONONONONOYESNONONONONONONONONONON...
x3      dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=MZ...
x4      GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=hC...
                              ...                        
x141    NONONONONONONONONONONONONONONONONONONONONONONO...
x142    YESYESNOYESNOYESYESNOYESYESNOYESYESYESNOYESNOY...
x143                                                36674
x144                                          5194.296623
x145                                          5050.337816
Length: 146, dtype: object
Processed Features Shape: (9999, 34855)
   id        x5        x6        x7        x8        x9  x15       x16  x17  \
0   1  0.576561  0.073139  0.481394  0.115697  0.472474   42  0.396065    3   
1   2  0.000000  0.000000  0.000000  0.000000  0.000000    0  0.000000    0   
2   3  1.341803  0.051422  0.935572  0.041440  0.501710   

In [8]:
#2_Feature_Engineering(Bag_Of_Words):

from sklearn.feature_extraction.text import CountVectorizer

# Sample text data for illustration (assuming we have a 'text' column in train_features)
sample_texts = ["this is a sample", "this is another example", "sample text data"]

# Initialize CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the sample text data
bow_features = count_vectorizer.fit_transform(sample_texts).toarray()

# Convert to DataFrame
bow_df = pd.DataFrame(bow_features, columns=count_vectorizer.get_feature_names_out())

# Display the BoW features
print(bow_df)


   another  data  example  is  sample  text  this
0        0     0        0   1       1     0     1
1        1     0        1   1       0     0     1
2        0     1        0   0       1     1     0


In [9]:
pip install scipy==1.12

Note: you may need to restart the kernel to use updated packages.


In [10]:
#Word_Embeddings:
import numpy as np
import gensim as gsm

# Example text corpus (tokenized)
corpus = [["this", "is", "a", "sample"], ["this", "is", "another", "example"], ["sample", "text", "data"]]

# Train a Word2Vec model
word2vec_model = gsm.models.Word2Vec(sentences=corpus, vector_size=50, window=5, min_count=1, workers=4)

# Function to get average word vectors for a text
def get_average_word2vec(text, model, vector_size):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

# Get Word2Vec features for each sample text
word2vec_features = np.array([get_average_word2vec(text, word2vec_model, 50) for text in sample_texts])

# Convert to DataFrame
word2vec_df = pd.DataFrame(word2vec_features)

# Display the Word2Vec features
print(word2vec_df)


         0         1         2         3         4         5         6   \
0 -0.006519  0.004237 -0.001714  0.002955  0.000257 -0.008938  0.007040   
1 -0.011792  0.007407 -0.003109 -0.001603  0.005724 -0.010330  0.003259   
2  0.004917 -0.004131 -0.001277  0.007402 -0.001716  0.005741  0.007864   

         7         8         9   ...        40        41        42        43  \
0  0.008833 -0.002910 -0.002415  ... -0.003750 -0.001428 -0.007618 -0.003546   
1  0.000938 -0.003392 -0.004002  ...  0.004225  0.002034  0.004890 -0.003521   
2  0.012101 -0.009363  0.006768  ... -0.009407  0.003441 -0.004974 -0.003653   

         44        45        46        47        48        49  
0  0.008083  0.006100 -0.002383 -0.003210  0.002444  0.005933  
1  0.010903  0.010580  0.002886 -0.005228  0.003749 -0.000603  
2  0.000814  0.004598 -0.000587  0.003016 -0.005930  0.008471  

[3 rows x 50 columns]


In [11]:
#TF_IDF_Vectorization:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the sample text data
tfidf_features = tfidf_vectorizer.fit_transform(sample_texts).toarray()

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_features, columns=tfidf_vectorizer.get_feature_names_out())

# Display the TF-IDF features
print(tfidf_df)


    another      data   example        is   sample      text      this
0  0.000000  0.000000  0.000000  0.577350  0.57735  0.000000  0.577350
1  0.562829  0.000000  0.562829  0.428046  0.00000  0.000000  0.428046
2  0.000000  0.622766  0.000000  0.000000  0.47363  0.622766  0.000000


In [12]:
#Combinig_All_Features:
# Assuming 'train_features' has a 'text' column containing the raw text data
print(train_features)
train_features['text'] = pd.Series(sample_texts)  # This line is for illustration purposes

# Generate BoW features for the actual text data df['Review'].values.astype('U')
bow_features = count_vectorizer.fit_transform(train_features['text'].values.astype('U')).toarray()
bow_df = pd.DataFrame(bow_features, columns=count_vectorizer.get_feature_names_out())

# Generate TF-IDF features for the actual text data
tfidf_features = tfidf_vectorizer.fit_transform(train_features['text'].values.astype('U')).toarray()
tfidf_df = pd.DataFrame(tfidf_features, columns=tfidf_vectorizer.get_feature_names_out())

# Generate Word2Vec features for the actual text data
word2vec_features = np.array([get_average_word2vec(text, word2vec_model, 50) for text in train_features['text'].fillna("")])
word2vec_df = pd.DataFrame(word2vec_features)

# Combine all features
combined_features = pd.concat([processed_features.reset_index(drop=True), bow_df, tfidf_df, word2vec_df], axis=1)

print("Combined Features Shape:", combined_features.shape)
print(combined_features.head())


        id   x1   x2                                            x3  \
0        1   NO   NO  dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=   
1        2   NO   NO  MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=   
2        3   NO   NO  ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=   
3        4  YES   NO  BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=   
4        5   NO   NO  RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=   
...    ...  ...  ...                                           ...   
9994  9995   NO   NO  jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=   
9995  9996   NO   NO  Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=   
9996  9997  YES  YES  9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=   
9997  9998   NO   NO  9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=   
9998  9999  YES  YES  MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=   

                                                x4        x5        x6  \
0     GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=  0.576561  0.073139   
1     hCXwO

In [13]:
#3 Model selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Split the data
X_train, X_val, y_train, y_val = train_test_split(combined_features, train_features, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
logistic_model = MultiOutputClassifier(LogisticRegression(max_iter=1000))

X_train.columns  = X_train.columns.astype(str)
y_train.columns  = y_train.columns.astype(str)

# Train the model
logistic_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = logistic_model.predict(X_val)

# Evaluate the model
print(classification_report(y_val, y_pred))

ValueError: Input contains NaN